In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score,roc_curve
from matplotlib import pyplot as plt
from sklearn.model_selection import GridSearchCV

In [6]:
diabetes = pd.read_csv('diabetes.csv')
df = diabetes.copy()
y = df['Outcome']
X = df.drop(['Outcome'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [7]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier().fit(X_train, y_train)

In [8]:
y_pred = xgb_model.predict(X_test)
accuracy_score(y_test, y_pred)

0.7142857142857143

In [9]:
?xgb_model

Type:        XGBClassifier
String form:
XGBClassifier(base_score=None, booster=None, callbacks=None,
           colsample_bylevel=None <...> , multi_strategy=None, n_estimators=None,
           n_jobs=None, num_parallel_tree=None, ...)
File:        c:\users\lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py
Docstring:  
Implementation of the scikit-learn API for XGBoost classification.
See :doc:`/python/sklearn_estimator` for more information.

Parameters
----------

    n_estimators : Optional[int]
        Number of boosting rounds.

    max_depth :  typing.Optional[int]

        Maximum tree depth for base learners.

    max_leaves : typing.Optional[int]

        Maximum number of leaves; 0 indicates no limit.

    max_bin : typing.Optional[int]

        If using histogram-based algorithm, maximum number of bins per feature

    grow_policy : typing.Optional[str]

        Tree growing policy.

        - depthwise: Favors splitting at nodes closest to the node,
        - lossguide:

In [14]:
xgb_params = {
        'n_estimators': [100, 500, 1000, 2000],
        'subsample': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5,6],
        'learning_rate': [0.1,0.01,0.02,0.05]}

In [15]:
xgb = XGBClassifier()

xgb_cv = GridSearchCV(xgb, xgb_params, cv=10, n_jobs=-1, verbose=2)

In [16]:
xgb_cv_model = xgb_cv.fit(X_train, y_train)

Fitting 10 folds for each of 192 candidates, totalling 1920 fits


c:\Users\Lenovo\anaconda3\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [17]:
xgb_cv_model.best_params_

{'learning_rate': 0.02, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}

In [18]:
xgb_tuned = XGBClassifier(n_estimators= xgb_cv_model.best_params_['n_estimators'],
                          max_depth= xgb_cv_model.best_params_['max_depth'],
                          learning_rate= xgb_cv_model.best_params_['learning_rate'],
                          subsample= xgb_cv_model.best_params_['subsample']).fit(X_train, y_train)

In [19]:
y_pred_tuned = xgb_tuned.predict(X_test)
accuracy_score(y_test, y_pred_tuned)

0.7575757575757576